<a href="https://colab.research.google.com/github/kameoman/pythonlesson/blob/main/python%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##BeautifulSoupでスクレイピング

In [1]:
import requests

from bs4 import BeautifulSoup

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)


In [7]:
query = "橋本環奈"
url = "https://www.google.com/search?q={}&hl=ja&tbm=isch".format(query)
url

'https://www.google.com/search?q=橋本環奈&hl=ja&tbm=isch'

In [8]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# すべての要素が読み込まれるまで待つ。タイムアウトは15秒。
WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located)

In [9]:
driver.get(url)
html = driver.page_source.encode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [ ]:
soup

In [ ]:
soup.find_all("img")

In [12]:
img_tags = soup.find_all("img")

In [13]:
len(img_tags)

83

In [ ]:
img_urls = []

for img_tag in img_tags:
  url = img_tag.get("src")
  

  if url is None:
    url = img_tag.get("data-src")

  if url is not None:
    img_urls.append(url)
    print(url)

In [15]:
def download_image(url, file_path):
  r = requests.get(url, stream=True)

  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)

In [16]:
import base64

def save_base64_image(data, file_path):
  # base64の読み込みは4文字ごとに行う。4文字で区切れない部分は「=」で補う
  data = data + '=' * (-len(data) % 4)
  img = base64.b64decode(data.encode())
  with open(file_path, "wb") as f:
      f.write(img)

In [18]:
import os
import re

# ご自分の環境に合わせて任意のディレクトリパスを指定してください。
save_dir = "/content/drive/MyDrive/photo/python/橋本環奈"

os.makedirs(save_dir, exist_ok=True)

base64_string = "data:image/jpeg;base64,"

# png画像も対象にする（動画公開後に追記してます）
png_base64_string = "data:image/png;base64,"


for index, url in enumerate(img_urls):
  # enumerateを使えばリストのindexを取得できます。このindexをそのままファイル名にします
  # formatを使えば文字列内の指定した場所に変数の内容を入れることができます
  file_name = "{}.jpg".format(index)
  # print(file_name)
  # print(url)

  image_path = os.path.join(save_dir, file_name)

  if len(re.findall(base64_string, url)) > 0 or len(re.findall(png_base64_string, url)) > 0:
    url = url.replace(base64_string, "")
    save_base64_image(data=url, file_path=image_path)
  else:
    download_image(url=url, file_path=image_path)